In [2]:
from pyspark import SparkContext

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
! conda install -c conda-forge pyspark 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/yas/anaconda3

  added / updated specs:
    - pyspark


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    py4j-0.10.7                |             py_1         177 KB  conda-forge
    pyspark-2.4.5              |             py_0       207.0 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:       210.2 MB

The following NEW packages will be INSTALLED:

  py4j               conda-forge/noarch::py4j-0.10.

In [ ]:
y

In [ ]:
!y